In [3]:
# Includes some packages and self written code
using LinearAlgebra, TensorOperations, KrylovKit
using JLD

include("qlm_mpo.jl");
include("func_dmrg.jl");


# Squared Ice - MPS

This notebook implements the Squared Ice Model in a DMRG procedure

## Hamiltonian and grid definition
We define the Square Ice Hamiltonian

\begin{equation}
    H = \sum_\square (-f_\square + \lambda f^2_\square)\,,\label{eq:hamiltonian}
\end{equation}
where we some over all plaquettes. The plaquette operator is defined as:
\begin{equation}\label{eq:plaquette}
       f_\square = \sigma^+_{\mu_1}\sigma^+_{\mu_2}\sigma^-_{\mu_3}\sigma^-_{\mu_4}\, + h.c..
\end{equation}
The Hamiltonian eq.\eqref{eq:hamiltonian} is invariant under the local symmetry:
\begin{equation}\label{eq:}
    G_\nu = \sum_{\hat{i}\in\{\hat{x},\hat{y}\}} ( \sigma_{\nu-\hat{i}/2}-\sigma_{\nu+\hat{i}/2})
\end{equation}
which counts the difference between in and outgoing arrows at vertex $\nu$.
In this work 
\begin{equation}\label{eq:ice_rule}
G_\nu=0 \qquad\text{ for all }\qquad \nu \in \Omega
\end{equation}

For the cylindrical lattice:
\begin{align}\label{eq:geometry}
\Omega=\{\nu = (n,m) |\qquad n\in\{1,\dots,L_x\}, \;  m \in\{1,\dots L_y\}\}
\end{align}
In all our lattices $L_y \ll L_x$. For now we fix $L_y = 2$.


In [9]:
NStates=16   # is the number of states of the local hilbert state
BondDim= 40  # is the inner bond dimension
Lx = 9  # is the number of sites alonge the QL chain

9

In [10]:
"""
randmps(physdims::NTuple{N,Int}, Dmax::Int, [T::Type{<:Number} = Float64])
randmps(N::Int, d::Int, Dmax::Int, [T::Type{<:Number} = Float64])

Construct a random right canonical MPS for a system with `N`, where site `n` has local Hilbert
space dimension `physdims[n]` (first method) or `d` (second method), and the maximal bond
dimension is `Dmax`. Entries of the MPS tensors will be of type `T`, defaulting to `Float64`.
"""
function randmps(physdims::Dims{N}, Dmax::Int, T::Type{<:Number} = Float64) where {N}
    bonddims = Vector{Int}(undef, N+1)
    bonddims[1] = 1
    bonddims[N+1] = 1
    Nhalf = div(N,2)
    for n = 2:N
        bonddims[n] = min(Dmax, bonddims[n-1]*physdims[n-1])
    end
    for n = N:-1:1
        bonddims[n] = min(bonddims[n], bonddims[n+1]*physdims[n])
    end

    As = Vector{Any}(undef, N)
    for n = 1:N
        d = physdims[n]
        Dl = bonddims[n]
        Dr = bonddims[n+1]
        As[n] = reshape(randisometry(T, Dl, d*Dr), (Dl, d, Dr))
    end
    return As
end
randmps(N::Int, d::Int, Dmax::Int, T = Float64) = randmps(ntuple(n->d, N), Dmax, T)

randmps (generic function with 4 methods)

In [14]:

A = randmps(Lx, Nstates, BondDim);


#M =  mpoqlm_fixed_with_interaction(N ; coupling=-0.0  );
#E, A, F = dmrgconvergence!(A, M ; verbose = true);

UndefVarError: UndefVarError: Nstates not defined

In [16]:
size(A)

(3,)